# Installing requirements

In [48]:
dataset_path = 'dataset.json'
batch_dataset_path = 'batch_dataset.json'
seed = 0

In [49]:
!pip install huggingface_hub
!pip install datasets
!pip install scikit-learn
!pip install matplotlib

from pathlib import Path
from huggingface_hub import notebook_login
import datasets
from tqdm import tqdm
import numpy as np
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer
import random
import matplotlib.pyplot as plt

You should consider upgrading via the 'C:\Users\Homeworkace\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Homeworkace\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Homeworkace\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Homeworkace\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Download the dataset

In [4]:
if Path(dataset_path).exists() :
    dataset = pd.read_json(dataset_path)
else :
    notebook_login()
    hugging_face_dataset = datasets.load_dataset('lmsys/chatbot_arena_conversations')
    dataset = hugging_face_dataset['train'].to_pandas()
    dataset.to_json(dataset_path)

In [5]:
dataset[dataset['turn'] > 3][:1]

,question_id,model_a,model_b,winner,judge,conversation_a,conversation_b,turn,anony,language,tstamp,openai_moderation,toxic_chat_tag
33,8120899314f74641b09c2aa114d4d253,alpaca-13b,vicuna-13b,model_b,arena_user_316,[{'content': 'Salut ! Comment ça va ce matin ?...,[{'content': 'Salut ! Comment ça va ce matin ?...,6,True,French,1.682354e+09,"{'categories': {'harassment': False, 'harassme...","{'roberta-large': {'flagged': False, 'probabil..."


# Create text-author pairs

In [6]:
# Initialize an empty list for processed data
text_author_pairs = []

# Iterate through each row of the DataFrame
for _, row in dataset.iterrows():
    # Process conversation_a (assistant role)
    for message in row["conversation_a"]:
        if message["role"] == "assistant":
            text_author_pairs.append({
                "text": message["content"],
                "author": row["model_a"]
            })

    # Process conversation_b (assistant role)
    for message in row["conversation_b"]:
        if message["role"] == "assistant":
            text_author_pairs.append({
                "text": message["content"],
                "author": row["model_b"]
            })

    # Process user messages (common across both conversations)
    for message in row["conversation_a"]:  # Check only `conversation_a` since user messages are identical
        if message["role"] == "user":
            text_author_pairs.append({
                "text": message["content"],
                "author": "human"
            })

# Create token count vectors for each text

In [136]:
# Extract texts and their authors
texts = [entry['text'] for entry in text_author_pairs]
authors = [entry['author'] for entry in text_author_pairs]

# Vectorize the paragraphs
vectoriser = CountVectorizer()
text_vectors = vectoriser.fit_transform(texts)  # Each row is a text

# Allocate quotas for each batch.
batch_count = {} 
for author in list(set(authors)) :
    # Find out how many batches of 200 texts are needed.
    number_of_batches = round(authors.count(author) / 200)
    for i in range(number_of_batches) :
        # Distribute the remainder/shortfall evenly.
        batch_count[str(i) + '_' + author] = np.floor((i + 1) * authors.count(author) / number_of_batches)
        for j in range(i) :
            # Subtract the count of previous batches.
            batch_count[str(i) + '_' + author] -= batch_count[str(j) + '_' + author]
        
print('Populating batches...')
batch_vectors = {}
batch_quota = batch_count.copy()
for i, author in enumerate(tqdm(authors)):
    batch_to_insert = [name for name in batch_quota if author in name and batch_quota[name] > 0][0]
    if batch_to_insert in batch_vectors:
        # The resulting vectors represent the number of times each token appears in that author's entire corpus.
        batch_vectors[batch_to_insert] += text_vectors[i].toarray()
    else:
        # Add this new batch to the list.
        batch_vectors[batch_to_insert] = text_vectors[i].toarray()
    batch_quota[batch_to_insert] -= 1

# Convert to a dataframe
print('Converting into DataFrame...')
batch_dataset = pd.DataFrame({name: vector.flatten() for name, vector in tqdm(batch_vectors.items())}, index = vectoriser.get_feature_names_out())

batch_dataset

Populating batches...


100%|████████████████████████████████████████████████████████████████████████| 117948/117948 [01:21<00:00, 1442.31it/s]


Converting into DataFrame...


100%|██████████████████████████████████████████████████████████████████████████████| 591/591 [00:00<00:00, 2221.29it/s]


,0_chatglm-6b,0_koala-13b,0_human,0_oasst-pythia-12b,0_alpaca-13b,0_vicuna-13b,0_dolly-v2-12b,0_stablelm-tuned-alpha-7b,0_llama-13b,1_human,...,190_human,191_human,192_human,6_wizardlm-13b,193_human,6_gpt4all-13b-snoozy,5_guanaco-33b,194_human,195_human,196_human
00,0,0,0,0,8,3,1,0,1,3,...,0,0,0,3,0,0,0,0,0,2
000,3,1,0,4,0,2,4,0,0,0,...,0,0,3,6,0,3,0,7,2,0
0000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000000,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ｔｏ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ｗｅｅｋｓ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𝘀𝗶𝗺𝗽𝗹𝗲,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
𝘀𝘂𝗽𝗲𝗿𝗵𝘂𝗺𝗮𝗻,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Group texts by author

In [137]:
# Find the average text length of each author. 
batch_average_length = {name: np.sum(vector) / batch_count[name] for name, vector in batch_vectors.items()}

# Normalise the length of each corpus.
print('Normalising...')
for name in tqdm(batch_vectors) :
    batch_dataset[name] /= np.sum(batch_vectors[name])
    
# Since it has been lost, add the average text length as a separate feature.
batch_dataset = pd.concat([batch_dataset, pd.DataFrame(batch_average_length, index = [' length'])])

batch_dataset

Normalising...


100%|████████████████████████████████████████████████████████████████████████████████| 591/591 [01:13<00:00,  8.04it/s]


,0_chatglm-6b,0_koala-13b,0_human,0_oasst-pythia-12b,0_alpaca-13b,0_vicuna-13b,0_dolly-v2-12b,0_stablelm-tuned-alpha-7b,0_llama-13b,1_human,...,190_human,191_human,192_human,6_wizardlm-13b,193_human,6_gpt4all-13b-snoozy,5_guanaco-33b,194_human,195_human,196_human
00,0.000000,0.000000,0.000000,0.000000,0.000926,0.000135,0.000083,0.000000,0.000095,0.000755,...,0.00,0.000,0.000000,0.000124,0.000,0.000000,0.000000,0.000000,0.000000,0.000241
000,0.000095,0.000041,0.000000,0.000201,0.000000,0.000090,0.000331,0.000000,0.000000,0.000000,...,0.00,0.000,0.000619,0.000248,0.000,0.000157,0.000000,0.000939,0.000454,0.000000
0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
000000,0.000000,0.000082,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ｗｅｅｋｓ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
𝘀𝗶𝗺𝗽𝗹𝗲,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
𝘀𝘂𝗽𝗲𝗿𝗵𝘂𝗺𝗮𝗻,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
𝘀𝘂𝗽𝗲𝗿𝗵𝘂𝗺𝗮𝗻𝘀,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000


In [143]:
batch_dataset.to_json(batch_dataset_path)

# Custom holdout

In [139]:
def holdout(df, authors) :
    training_df = df.copy()
    holdout_list = []
    for author in authors :
        batch = training_df[random.choice([batch for batch in training_df.columns if author in batch])]
        training_df.drop(batch.name, axis = 1)
        holdout_list.append(batch)

    holdout_df = pd.concat(holdout_list, axis = 1)
    return training_df, holdout_df

In [140]:
random.seed(seed)
training_dataset, holdout_dataset = holdout(batch_dataset, list(set(authors)))

holdout_dataset

,27_gpt-3.5-turbo,12_vicuna-7b,13_chatglm-6b,1_alpaca-13b,8_mpt-7b-chat,15_stablelm-tuned-alpha-7b,12_gpt-4,25_oasst-pythia-12b,9_claude-v1,30_koala-13b,...,4_gpt4all-13b-snoozy,3_llama-13b,32_vicuna-13b,4_claude-instant-v1,9_dolly-v2-12b,4_RWKV-4-Raven-14B,0_guanaco-33b,8_palm-2,17_fastchat-t5-3b,5_wizardlm-13b
00,0.000,0.000443,0.000303,0.000000,0.000000,0.000000,0.000071,0.000000,0.000325,0.000000,...,0.000000,0.000336,0.000000,0.000000,0.000000,0.000000,0.000000,0.000061,0.000046,0.000000
000,0.000,0.000000,0.000000,0.000261,0.000061,0.000132,0.000000,0.000123,0.000909,0.000244,...,0.000192,0.000112,0.000039,0.000111,0.000290,0.000179,0.000031,0.000183,0.000092,0.000089
0000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
00000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000044,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ｗｅｅｋｓ,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
𝘀𝗶𝗺𝗽𝗹𝗲,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
𝘀𝘂𝗽𝗲𝗿𝗵𝘂𝗺𝗮𝗻,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
𝘀𝘂𝗽𝗲𝗿𝗵𝘂𝗺𝗮𝗻𝘀,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Feature selection

In [141]:
human_dataset = training_dataset[[name for name in training_dataset.columns if 'human' in name]].copy()
llm_dataset = training_dataset[[name for name in training_dataset.columns if not 'human' in name]].copy()
bruh = llm_dataset[llm_dataset.ne(0).sum(axis=1) == llm_dataset.shape[1]]

#training_dataset

# Now put `human_dataset` back into `training_dataset`.

In [142]:
s1 = pd.merge(bruh, human_dataset, left_index=True, right_index=True, validate='1:1')
s1

,0_chatglm-6b,0_koala-13b,0_oasst-pythia-12b,0_alpaca-13b,0_vicuna-13b,0_dolly-v2-12b,0_stablelm-tuned-alpha-7b,0_llama-13b,1_vicuna-13b,1_koala-13b,...,187_human,188_human,189_human,190_human,191_human,192_human,193_human,194_human,195_human,196_human
10,0.000698,0.000945,0.000705,0.000463,0.000630,0.000994,0.000687,0.001040,0.000472,0.000806,...,0.001238,0.000262,0.001459,0.001037,0.001772,0.001856,0.002592,0.001610,0.000227,0.000481
about,0.001776,0.001068,0.001309,0.000810,0.002116,0.001160,0.003070,0.002459,0.001769,0.002110,...,0.001856,0.003148,0.005209,0.005965,0.002658,0.005568,0.003672,0.004428,0.002496,0.001443
after,0.000317,0.000205,0.000554,0.000231,0.000180,0.000580,0.000404,0.000095,0.000708,0.000345,...,0.000619,0.000262,0.000417,0.000259,0.001624,0.000825,0.000864,0.005233,0.000227,0.000601
all,0.001237,0.001068,0.001107,0.001620,0.000856,0.000994,0.002101,0.001797,0.000708,0.001189,...,0.003249,0.001574,0.001459,0.002075,0.002805,0.002475,0.002592,0.002147,0.001815,0.000241
also,0.003075,0.002998,0.002718,0.004165,0.003062,0.002486,0.002747,0.002838,0.002713,0.003108,...,0.000309,0.001049,0.000417,0.000259,0.000886,0.000412,0.001080,0.001342,0.000908,0.000241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
world,0.000919,0.001027,0.000856,0.001620,0.001396,0.000994,0.001656,0.002459,0.001455,0.001650,...,0.000464,0.000787,0.000417,0.000259,0.000000,0.000825,0.001944,0.000000,0.000681,0.000361
would,0.000476,0.001930,0.001057,0.003008,0.001891,0.001740,0.002141,0.001419,0.001455,0.001919,...,0.001238,0.001049,0.003751,0.003112,0.002362,0.003300,0.001080,0.001476,0.002042,0.000722
you,0.009480,0.010103,0.012080,0.010066,0.008240,0.010108,0.013492,0.013432,0.008965,0.009247,...,0.008199,0.017576,0.012294,0.013485,0.005758,0.016498,0.013610,0.013149,0.010211,0.003007
your,0.003234,0.004271,0.006845,0.003587,0.004233,0.002900,0.004039,0.003689,0.003146,0.005986,...,0.001547,0.002099,0.003126,0.002334,0.002658,0.003300,0.006697,0.002549,0.003177,0.000962


# Centering

In [94]:
features_mean = {feature: np.mean(common_tokens_only.loc[feature]) for feature in common_tokens_only.index}
for feature in common_tokens_only.index :
    common_tokens_only.loc[feature] -= features_mean[feature]

common_tokens_only

C:\Users\Homeworkace\AppData\Local\Temp\ipykernel_3508\2290679722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_tokens_only.loc[feature] -= features_mean[feature]


,chatglm-6b,koala-13b,human,oasst-pythia-12b,alpaca-13b,vicuna-13b,dolly-v2-12b,stablelm-tuned-alpha-7b,llama-13b,fastchat-t5-3b,...,gpt-4,RWKV-4-Raven-14B,claude-v1,mpt-7b-chat,palm-2,claude-instant-v1,vicuna-7b,wizardlm-13b,gpt4all-13b-snoozy,guanaco-33b
04,-1.659460e-05,-1.370296e-05,1.677537e-04,0.000006,-8.678607e-06,-1.157621e-05,0.000011,-0.000013,-0.000013,-0.000021,...,-1.315954e-05,-0.000016,-0.000009,-0.000003,-0.000009,-1.371759e-05,-1.606912e-05,-8.426521e-06,0.000012,-3.784737e-06
07,-1.010447e-05,-1.778170e-05,1.316913e-04,-0.000004,-2.346519e-07,-9.942558e-06,0.000063,-0.000002,-0.000012,-0.000018,...,-1.413636e-05,-0.000016,-0.000017,-0.000010,-0.000016,-6.344560e-06,-1.330445e-05,1.031573e-05,-0.000020,-3.276912e-06
10th,-1.977722e-06,6.398017e-06,3.594102e-06,-0.000001,1.979629e-05,-7.444931e-06,-0.000007,0.000014,0.000018,-0.000002,...,-3.040310e-06,-0.000006,0.000002,-0.000008,-0.000006,-5.062149e-07,-7.434545e-06,-5.870088e-06,0.000012,-1.194823e-05
121,-7.249268e-06,-3.510167e-06,-1.015076e-06,-0.000006,-5.275304e-06,-3.614247e-06,-0.000004,-0.000007,0.000029,-0.000002,...,8.572468e-06,-0.000005,-0.000006,-0.000006,-0.000009,-2.378161e-06,-2.160005e-07,-3.165229e-06,0.000015,2.435276e-05
123,-7.138689e-06,-9.082713e-06,2.128950e-05,0.000002,-9.173066e-06,-4.725128e-06,0.000014,-0.000009,0.000036,0.000023,...,-2.262665e-06,0.000008,-0.000002,-0.000010,0.000003,-7.955585e-06,-1.259551e-05,-1.103106e-05,-0.000017,-5.910486e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xml,2.507718e-06,-1.130076e-05,-1.435235e-05,-0.000020,-2.744408e-05,-4.982951e-06,-0.000020,-0.000007,-0.000028,-0.000025,...,-1.014111e-05,-0.000012,0.000005,-0.000035,-0.000016,1.339616e-04,6.617769e-05,-2.322393e-05,0.000005,5.420947e-05
yield,-4.068175e-07,-7.484136e-09,1.273093e-05,0.000004,-6.449536e-06,-5.969662e-06,0.000002,0.000013,-0.000003,-0.000018,...,9.467603e-07,0.000011,0.000017,-0.000005,-0.000016,1.826071e-05,-2.555835e-06,-6.197449e-06,-0.000010,-7.155021e-06
yo,-2.264166e-05,7.432457e-05,1.354101e-07,-0.000019,-3.063821e-05,8.175055e-06,0.000022,-0.000014,0.000246,-0.000032,...,-2.124439e-05,0.000008,-0.000016,-0.000010,-0.000025,-2.087585e-05,-3.009259e-05,-3.460627e-05,-0.000026,-2.900692e-05
yoga,8.011033e-06,2.336677e-05,-1.242676e-05,0.000010,-2.040026e-06,1.009045e-07,-0.000018,-0.000002,-0.000010,0.000012,...,2.365800e-05,-0.000012,0.000003,-0.000004,-0.000008,-4.181869e-06,-4.371233e-06,1.247842e-05,-0.000002,-1.231293e-05


# Standardisation

In [95]:
features_std = {feature: np.std(common_tokens_only.loc[feature]) for feature in common_tokens_only.index}
for feature in common_tokens_only.index :
    common_tokens_only.loc[feature] /= features_std[feature]

common_tokens_only

C:\Users\Homeworkace\AppData\Local\Temp\ipykernel_3508\1559195010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  common_tokens_only.loc[feature] /= features_std[feature]


,chatglm-6b,koala-13b,human,oasst-pythia-12b,alpaca-13b,vicuna-13b,dolly-v2-12b,stablelm-tuned-alpha-7b,llama-13b,fastchat-t5-3b,...,gpt-4,RWKV-4-Raven-14B,claude-v1,mpt-7b-chat,palm-2,claude-instant-v1,vicuna-7b,wizardlm-13b,gpt4all-13b-snoozy,guanaco-33b
04,-0.431668,-0.356449,4.363698,0.146259,-0.225753,-0.301126,0.274759,-0.331525,-0.337875,-0.535411,...,-0.342313,-0.425501,-0.242503,-0.086902,-0.227190,-0.356829,-0.417998,-0.219195,0.308310,-0.098451
07,-0.296628,-0.522002,3.865948,-0.127749,-0.006888,-0.291875,1.848830,-0.052441,-0.366397,-0.516423,...,-0.414989,-0.465286,-0.506684,-0.285628,-0.473377,-0.186252,-0.390567,0.302830,-0.589324,-0.096197
10th,-0.219408,0.709794,0.398729,-0.162643,2.196195,-0.825938,-0.749657,1.572942,2.044318,-0.223530,...,-0.337291,-0.622410,0.235980,-0.942968,-0.668273,-0.056159,-0.824786,-0.651226,1.373226,-1.325533
121,-0.715568,-0.346485,-0.100197,-0.567509,-0.520720,-0.356759,-0.400016,-0.654511,2.835518,-0.177021,...,0.846180,-0.495333,-0.634532,-0.572015,-0.912404,-0.234746,-0.021321,-0.312437,1.488828,2.403837
123,-0.541378,-0.688808,1.614537,0.159875,-0.695660,-0.358341,1.064464,-0.703104,2.733777,1.715756,...,-0.171594,0.625077,-0.123337,-0.774479,0.200138,-0.603329,-0.955209,-0.836565,-1.297514,-0.448235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xml,0.065113,-0.293423,-0.372658,-0.510391,-0.712583,-0.129382,-0.514301,-0.172432,-0.721470,-0.660726,...,-0.263313,-0.315248,0.141204,-0.918644,-0.405883,3.478302,1.718298,-0.603007,0.134076,1.407544
yield,-0.041512,-0.000764,1.299084,0.444663,-0.658121,-0.609154,0.245905,1.326353,-0.323061,-1.872844,...,0.096609,1.145220,1.717828,-0.465320,-1.671329,1.863352,-0.260801,-0.632398,-1.045410,-0.730110
yo,-0.376003,1.234283,0.002249,-0.313584,-0.508799,0.135760,0.373543,-0.227595,4.091339,-0.533455,...,-0.352799,0.126980,-0.270827,-0.173829,-0.410719,-0.346678,-0.499738,-0.574695,-0.440035,-0.481708
yoga,0.723058,2.109033,-1.121612,0.906206,-0.184128,0.009107,-1.616724,-0.201847,-0.931317,1.072963,...,2.135320,-1.044680,0.288889,-0.371741,-0.725507,-0.377446,-0.394538,1.126275,-0.152961,-1.111338


In [96]:
print(np.mean(common_tokens_only.loc['_length']))
print(np.std(common_tokens_only.loc['_length']))

print(np.sum([np.std(common_tokens_only.loc[feature]) for feature in common_tokens_only.index]))

KeyError: '_length'

In [ ]:
# author_df.to_csv('bruh.csv')

In [ ]:
# Extracting the two features
x_token = 'zu'
y_token = '_length'

# Creating the scatter plot
feature_graph_figure = plt.figure(figsize=(8, 6))
feature_graph = feature_graph_figure.subplots()
feature_graph.scatter(common_tokens_only.loc[x_token], common_tokens_only.loc[y_token], color='blue', alpha=0.7)
feature_graph.set_xlabel("Feature '" + x_token + "'")
feature_graph.set_ylabel("Feature '" + y_token + "'")
feature_graph.grid(True)
feature_graph_figure.show()

In [ ]:
def power_iteration(A, num_iter=1000, tol=1e-7):
    """
    Power Iteration to compute the largest eigenvector of matrix A.
    Args:
        A (np.ndarray): Input symmetric matrix.
        num_iter (int): Number of iterations.
        tol (float): Tolerance for convergence.

    Returns:
        eigenvector (np.ndarray): Approximation of the largest eigenvector.
        eigenvalue (float): Corresponding eigenvalue.
    """
    b = np.random.rand(A.shape[1])  # Random initial vector
    b = b / np.linalg.norm(b)

    error_meter = tqdm(position = 0, total=7, bar_format='{bar} | {postfix}')
    for i in range(num_iter):
        b_next = A @ b  # Matrix-vector multiplication
        b_next = b_next / np.linalg.norm(b_next)  # Normalize
        
        # Check for convergence
        error = np.linalg.norm(b_next - b)
        b = b_next
        error_meter.update(-np.log10(error) - error_meter.n)
        error_meter.set_postfix_str('{:.2E}'.format(error) + ' of error after ' + str(i + 1) + ' iterations...')
        if error < tol:
            error_meter.close()
            break

    eigenvalue = b.T @ A @ b  # Rayleigh quotient for eigenvalue
    return b, eigenvalue

In [ ]:
def pca_power_iteration(data, num_components=2, num_iter=1000):
    """
    PCA using Power Iteration to compute the principal components.
    Args:
        data (np.ndarray): Input data matrix (rows are samples, columns are features).
        num_components (int): Number of principal components to compute.
        num_iter (int): Number of iterations for power iteration.

    Returns:
        components (list): List of principal components (eigenvectors).
        explained_variance (list): List of corresponding eigenvalues.
    """
    # Step 1: Center the data
    covariance_matrix = np.cov(data, rowvar=True, ddof = 0)
    
    # Compute covariance matrix manually
    n = data.shape[1]  # Number of samples (columns in your case)
    covariance_matrix = (data @ data.T) / n

    components = []
    explained_variance = []

    for i in range(num_components):
        # Step 2: Compute the largest eigenvector using power iteration
        eigenvector, eigenvalue = power_iteration(covariance_matrix, num_iter=num_iter)
        components.append(eigenvector)
        explained_variance.append(eigenvalue)
        
        # Step 3: Deflate the covariance matrix
        covariance_matrix -= eigenvalue * np.outer(eigenvector, eigenvector)
        if np.linalg.norm(covariance_matrix, ord='fro') < 1e-10 :
            print('Stopped as all variance has been extracted after computing ' + str(i + 1) + ' PCs.')
            break
        
    return components, explained_variance

In [ ]:
components, explained_variance = pca_power_iteration(common_tokens_only, num_components=20)

In [ ]:
print(len(explained_variance))
for variance in range(len(explained_variance)) :
    print(np.sum(explained_variance[:variance + 1]))
print(len(components[0]))
print(np.sum([np.std(common_tokens_only.loc[feature]) for feature in common_tokens_only.index]))

In [ ]:
# Creating the scatter plot
feature_graph_figure = plt.figure(figsize=(8, 6))
feature_graph = feature_graph_figure.subplots()
coords = [[np.dot(common_tokens_only[author], components[0]), np.dot(common_tokens_only[author], components[1])] for author in common_tokens_only]
feature_graph.scatter([coord[0] for coord in coords], [coord[1] for coord in coords], color='blue', alpha=0.7)
for i, author in enumerate(common_tokens_only.columns):
    feature_graph.annotate(author, coords[i])
feature_graph.set_xlabel("PC1")
feature_graph.set_ylabel("PC2")
feature_graph.grid(True)
feature_graph_figure.show()

In [ ]:
print(np.linalg.norm(common_tokens_only['claude-v1'] - common_tokens_only['claude-instant-v1']))
print(np.dot(common_tokens_only['claude-v1'], common_tokens_only['claude-instant-v1']))
print(np.linalg.norm(common_tokens_only['claude-v1'] - common_tokens_only['alpaca-13b']))
print(np.dot(common_tokens_only['claude-v1'], common_tokens_only['alpaca-13b']))
print(np.linalg.norm(common_tokens_only['alpaca-13b'] - common_tokens_only['palm-2']))
print(np.dot(common_tokens_only['alpaca-13b'], common_tokens_only['palm-2']))

In [ ]:
np.linalg.norm(common_tokens_only['claude-v1'] - common_tokens_only['llama-13b'])

In [ ]:
!pip freeze